## 활성화 함수 계층

### ReLU  계층

In [1]:
class Relu:
    def __init__(self):
        # mask는 T/F 넘파이 배열. 순전파 입력값 0 이하는 True, 그 이외는 False 유지.
        self.mask = None

    def forward(self, x):
        self.mask = (x <= 0)
        out = x.copy()
        out[self.mask] = 0

        return out

    def backward(self, dout):
        dout[self.mask] = 0
        dx = dout

        return dx

### Sigmoid 계층

In [2]:
class Sigmoid:
    def __init__(self):
        self.out = None

    def forward(self, x):
        # 순전파의 출력을 out에 보관. 역전파 계산 시 값을 사용
        out = sigmoid(x)
        self.out = out
        return out

    def backward(self, dout):
        dx = dout * (1.0 - self.out) * self.out

        return dx

### Affine 계층
**Affine**: 순전파 때 수행하는 행렬의 곱을 **어파인 변환**이라고 한다. 이를 수행하는 처리를 Affine 계층이라고 구현해놓았다.

In [3]:
# # 예시에서 데이터가 2개(N=2)라고 가정할 때의 Affine
# class Affine:
#     def __init__(self, W, b):
#         self.W =W
#         self.b = b
#         self.x = None
#         self.dW = None
#         self.db = None
        
#     def forward(self, x):
#         self.x = x
#         out = np.dot(x, self.W) + self.b
        
#         return out
    
#     def backward(self, dout):
#         dx = np.dot(dout, self.W.T)
#         self.dW = np.dot(self.x.T, dout)
#         self.db = np.sum(dout, axis=0)
        
#         return dx

In [4]:
# 입력 데이터가 텐서인 경우도 고려한 구현
class Affine:
    def __init__(self, W, b):
        self.W =W
        self.b = b
        
        self.x = None
        self.original_x_shape = None
        self.dW = None
        self.db = None

    def forward(self, x):
        self.original_x_shape = x.shape
        x = x.reshape(x.shape[0], -1)
        self.x = x

        out = np.dot(self.x, self.W) + self.b

        return out

    def backward(self, dout):
        dx = np.dot(dout, self.W.T)
        self.dW = np.dot(self.x.T, dout)
        self.db = np.sum(dout, axis=0)
        
        dx = dx.reshape(*self.original_x_shape)
        return dx

## 출력 계층

### Softmax - with - Loss 계층

In [5]:
class SoftmaxWithLoss:
    def __init__(self):
        # 손실
        self.loss = None
        # softmax의 출력
        self.y = None
        # 정답 라벨 (원-핫 벡터)
        self.t = None

    def forward(self, x, t):
        self.t = t
        self.y = softmax(x)
        self.loss = cross_entropy_error(self.y, self.t)
        
        return self.loss

    def backward(self, dout=1):
        batch_size = self.t.shape[0]
        if self.t.size == self.y.size:
            dx = (self.y - self.t) / batch_size
        else:
            dx = self.y.copy()
            dx[np.arange(batch_size), self.t] -= 1
            dx = dx / batch_size
        
        return dx